In [1]:
import pandas as pd
import numpy as np
import sys
import random

# spotipy dependencies
import spotipy
import spotipy.oauth2
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

# ML dependencies
import joblib
from joblib import load
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import IsolationForest

In [2]:
pd.set_option('display.max_rows', None)

# globals
client_id = 'BIGBOY_SECRET'
client_secret = 'SECRET_AF'
redirect_uri = 'http://localhost:8888/callback'
scope = 'user-top-read user-library-read playlist-modify-private playlist-modify-public'

# oauth2 login
sp_oauth = SpotifyOAuth(client_id = client_id,
                        client_secret = client_secret,
                        redirect_uri = redirect_uri,
                        scope = scope)
token_info = sp_oauth.get_cached_token() 
token = token_info['access_token']
sp = spotipy.Spotify(auth = token)

user_id = sp.current_user()['display_name']

In [68]:
# new login
if not token_info:
    auth_url = sp_oauth.get_authorize_url()
    print(auth_url)
    response = input('Paste the redirect url here: ')
    code = sp_oauth.parse_response_code(response)
    token_info = sp_oauth.get_access_token(code)
    token = token_info['access_token']
sp = spotipy.Spotify(auth=token)

https://accounts.spotify.com/authorize?client_id=84bb0f8b3cfa4cc69a8c5175ae89c6b5&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8888%2Fcallback&scope=user-top-read+user-library-read+playlist-modify-private+playlist-modify-public
Paste the redirect url here: http://localhost:8888/callback?code=AQA0Gq9XlfGEyHAFqu6wDig0BsB93zFv0STcU_jF4APIMMYJgoxMv2DmcupK413-KZCL-b-xJkr1QjfHjlPNRNDSAqeHSfkZpDPjtXjDmHhrqhP2BDT4YGYpCy_bAQG_k6EFAvEH1r5ngLuteZC6ti5C0xREvTapcBfSHumisHy9zfo1TccQmB9ScVp0OQzfXxqn2ztSKYBM_XdshCMrTJhYJeUYKhmxVOhhEWJZnMaOpsRVx8e9VG9G6a6CgH-UBCECyrTIGSB2bXP6keej_jHDLmhR


<ipython-input-68-0c810373b318>:7: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token_info = sp_oauth.get_access_token(code)


In [3]:
# utility functions

def login():
    sp_oauth = SpotifyOAuth(client_id = client_id,
                            client_secret = client_secret,
                            redirect_uri = redirect_uri,
                            scope = scope)
    token_info = sp_oauth.get_cached_token() 
    token = token_info['access_token']
    sp = spotipy.Spotify(auth = token)
    return sp

def refresh():
    global token_info, sp
    if sp_oauth.is_token_expired(token_info):
        token_info = sp_oauth.refresh_access_token(token_info['refresh_token'])
        token = token_info['access_token']
        sp = spotipy.Spotify(auth=token)

def random_id(playlist):
    random_seed = random.choice(range(len(playlist)))
    random_id = playlist.iloc[random_seed,[0]].values[0]
    return random_id

def diversity_score(playlist):
    diversity_score = round((playlist['artist'].nunique() / len(playlist['artist'])) *100, 2)
    return diversity_score

def rank(data):
    refresh()
    ranked = data.sort_values(by = ['scores'], ascending = False)
    return ranked[['scores', 'name', 'artist', 'album', 'track_id']]

In [4]:

def get_playlist_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

def getTrackFeatures(track_id):
    t_id = track_id
    meta = sp.track(track_id)
    features = sp.audio_features(track_id)

    # meta
    name = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    artist_id = meta['album']['artists'][0]['id']
    release_date = meta['album']['release_date']
    length = meta['duration_ms']
    popularity = meta['popularity']

    # features
    key = features[0]['key']
    mode = features[0]['mode']
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    valence = features[0]['valence']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    time_signature = features[0]['time_signature']

    track = [t_id,
             artist_id,
             name,
             album,
             artist,
             release_date,
             length,
             popularity,
             key,
             mode,
             acousticness,
             danceability,
             energy,
             valence,
             instrumentalness,
             liveness,
             loudness,
             speechiness,
             tempo,
             time_signature]
    return track

def parse_playlist_dataframe(dataframe):
    # pull ids from metadata
    track_list = []
    for i in range(len(dataframe)):
        ids = dataframe[i]['track']['id']
        track_list.append(ids)
    return track_list
     
    # parse features
    df = parse_features(track_list)
    return df

def parse_rec_id_list(track_id_list):
    # pull ids from metadata
    track_list = []
    for i in range(len(track_id_list['tracks'])):
        ids = track_id_list['tracks'][i]['id']
        track_list.append(ids)
    return track_list
     
    # parse features
    df = parse_features(track_list)
    return df


def parse_features(track_list):
    # parse features
    track_features = []
    for i in range(len(track_list)):
        track = getTrackFeatures(track_list[i])
        track_features.append(track)
    
    # build dataframe
    df = pd.DataFrame(track_features,
                      columns = ['track_id',
                                 'artist_id',
                                 'name',
                                 'album',
                                 'artist',
                                 'release_date',
                                 'length',
                                 'popularity',
                                 'key',
                                 'mode',
                                 'acousticness',
                                 'danceability',
                                 'energy',
                                 'valence',
                                 'instrumentalness',
                                 'liveness',
                                 'loudness',
                                 'speechiness',
                                 'tempo',
                                 'time_signature'])
    # squish some data
    df['popularity'] = df['popularity'] / 100
    df['length'] = (df['length'] / 1000) / 60
    return df

def parse_playlist(playlist_id):
    # pull and scrape metadata for track ids in playlist
    p_list = get_playlist_tracks(user_id, playlist_id)
    parsed_playlist = parse_playlist_dataframe(p_list)
    return parsed_playlist

def preprocess(df):
    preprocess = df.drop(['track_id',
                        'artist_id',
                        'name',
                        'album',
                        'artist',
                        'popularity',
                        'release_date',
                        'key',
                        'mode',
                        'tempo',
                        'time_signature',
                        'length'], axis = 1)
    
    # convert int64 to categoryical variables
    int64 = preprocess.select_dtypes('int64')
    for x in int64:
        preprocess[x] = preprocess[x].astype('category')
    return preprocess

# input (training_data)
# output (cleaned_og data)

def explorative_isolation_forest(training_data, cleaned_data):
    model = IsolationForest(n_estimators = 500,
                        max_samples = 'auto',
                        contamination = 0, # your tastes = 100% homogenous
                        random_state = 69)
    model.fit(training_data)
    cleaned_data['scores'] = model.decision_function(training_data)
    return cleaned_data

def create_new_model(data):
    refresh()
    predict_pipeline = make_pipeline(IsolationForest(n_estimators = (5000),
                                                     max_samples = 'auto',
                                                     max_features = 8,
                                                     contamination = 'auto',
                                                     n_jobs = -1,
                                                     random_state = 69)
                                    )
    processed = preprocess(data)
    predict_pipeline.fit(processed)
    joblib.dump(predict_pipeline, "classification.joblib")
    clf = load('classification.joblib')
    return print('New classification model created.')

def predict_scores(data):
    if 'scores' in data.columns:
        predictions = data.drop(['scores'], axis = 1)
    else:
        predictions = data
    clf = load('classification.joblib')
    processed = preprocess(predictions)
    scores = clf.decision_function(processed)
    predictions['scores'] = scores
    return predictions
      
def generate_playlist(input_track, input_playlist):
    refresh()
    pickiness = 4 # should be a user input
    knn = NearestNeighbors(n_neighbors = pickiness + 1)
    up_next = []
    up_next_length = 24 # should be a user input
    working_playlist = input_playlist
    
    seed = input_track
    up_next.append(input_track)
    
    for i in range(up_next_length): 

    # find seed track nearest neighbors (currently returns 6, trimmed down to 5)
        current_id = working_playlist.loc[working_playlist['track_id'] == seed]

    # retrain every, necessary evil to ensure closest remaining neighbors are selected
        knn_fit = preprocess(working_playlist)
        knn.fit(knn_fit)

    # find n nearest neighbors (6, generally includes self as nearest)
        processed_c_id = preprocess(current_id)
        neighbors = knn.kneighbors(processed_c_id,
                                   return_distance = False)
        neighbors_list = np.concatenate(neighbors).ravel().tolist()

    # exclude current seed as a next seed option
        seed_index = working_playlist.index[working_playlist['track_id'] == seed].to_list()
        for i in range(len(neighbors_list)-1):
            if neighbors_list[i] == seed_index[0]:
                del neighbors_list[i]
            else:
                pass

    # logic to reduce to drop furthest if seed isn't a nearest neighbor to self (not common in testing)
        if len(neighbors_list) == pickiness + 1:
            neighbors_list = neighbors_list[:pickiness]
        else:
            pass

    # randomly selected new seed from neighbors
        new_seed_index = []
        new_seed_index.append(random.choice(neighbors_list))
    
    # logic to greatly reduce probability of repeating artists back to back
        seed_artist = working_playlist.iloc[seed_index,[1]].values[0][0]
        new_seed_artist = working_playlist.iloc[new_seed_index,[1]].values[0][0]
        if seed_artist == new_seed_artist:
            neighbors_list.remove(new_seed_index[0])
            new_seed_index = []
            new_seed_index.append(random.choice(neighbors_list))
        else:
            pass
    
        new_seed = working_playlist.iloc[new_seed_index,[0]]
        new_seed_id = new_seed['track_id'].tolist()
        new_seed_id = new_seed_id[0]

    # update queue with new track id
        up_next.extend(new_seed['track_id'].tolist())

    # drop old seed from available seeds. cleans up dataframe
        working_playlist = working_playlist.drop(seed_index[0], 0)
        working_playlist.reset_index(inplace = True)
        working_playlist = working_playlist.drop(working_playlist.columns[[0]], 1)

        seed = new_seed_id

    # querry input playlist to generate a df with necessary info
    compile_playlist = []
    for i in range(len(up_next)):
        track = input_playlist.loc[input_playlist['track_id'] == up_next[i]]
        track_list = track[['track_id', 'name', 'artist','album']].values.tolist()
        compile_playlist.extend(track_list)
    final_playlist = pd.DataFrame(compile_playlist, columns =['track_id', 'name', 'artist','album'])
    
    return final_playlist

In [5]:
playlist_id = '6vqdAph2gxHHgvojxM7js4'

# https://open.spotify.com/playlist/6vqdAph2gxHHgvojxM7js4?si=7418ddda7d2c4b4d - melo death +
# https://open.spotify.com/playlist/4KOx5MotstO0KdKYxfeUKy?si=535182ac02934e00 - relax by wouldy

In [31]:
user_preference_playlist = parse_playlist(playlist_id)

In [32]:
playlist_features = parse_features(user_preference_playlist)
playlist_features.head()

,track_id,artist_id,name,album,artist,release_date,length,popularity,key,mode,acousticness,danceability,energy,valence,instrumentalness,liveness,loudness,speechiness,tempo,time_signature
0,7eLpkw9x3p88Fga5drnvqS,2J2V1PxHVZ1ANtT8Y2UdTS,Blah Loops,Songs Made While High,Like,2016-09-30,3.741667,0.27,1,1,0.458,0.688,0.743,0.572,0.002740,0.116,-6.880,0.111,88.513,4
1,2TTqgKHQwSkvxD6fI25nWK,0nvdwVbj7NT1WL9P8JowLD,Hands Up!,Hands Up!,blackwave.,2017-08-18,3.665000,0.13,0,1,0.297,0.620,0.443,0.581,0.000043,0.119,-8.145,0.375,87.423,4
2,6bvV5L5afKykg819xAIJWt,7bTL2or1gi4VAjY7XzoK8e,First Person,Time Being (Deluxe Edition),CYNE,2017-09-08,2.837933,0.48,1,1,0.507,0.686,0.800,0.467,0.000549,0.286,-7.279,0.216,96.480,4
3,6NXqSpbK8xkfOZRTVhC5nr,1mOiWC7OH9ANUtt3vd0A10,Smile,Smile,Shuko,2017-10-06,2.864883,0.00,9,1,0.103,0.551,0.675,0.374,0.000003,0.139,-4.668,0.345,101.952,5
4,5Wd3qtcrfnNc0zuQLbUsES,4pCZmnefAzcD4fVSkcVEYf,Polaroid,Polaroid,Melanin 9,2017-10-16,3.342317,0.00,7,1,0.694,0.749,0.828,0.495,0.000001,0.101,-7.785,0.322,77.648,4


In [16]:
diversity_score(analysis_results)

61.44

In [79]:
import cython
import eif
eif_playlist = playlist_features
input_data = preprocess(eif_playlist).to_numpy()
sample_size = len(input_data)
F1  = eif.iForest(input_data, ntrees=5000, sample_size=sample_size, ExtensionLevel=1)
S1 = F1.compute_paths(X_in=input_data)
eif_playlist['scores'] = 1 - S1
rank(eif_playlist)

,scores,name,artist,album,track_id
160,0.625061,Stoney,Saba,Bucket List Project,0HU8CeTestQjoRF2ekaLqx
28,0.624566,Suede,NxWorries,Yes Lawd!,7kFxZ5ckpMi2RhN1WH8cn2
46,0.624507,The Only One,Blu & Exile,Give Me My Flowers While I Can Still Smell Them,53GyUog0e8fbctqyJieLS2
37,0.623964,Alone,Kooley High,Heights,74ekur2cSq38RshtjzNMAX
179,0.622412,You Stressin',Bishop Nehru,You Stressin',60NwuLLD9fpWZcDIfjv1NS
9,0.621611,Elusive,blackwave.,Elusive,4mbOTHs6heygME9uDHg0mf
158,0.621559,U 47,Your Old Droog,Your Old Droog,1otYIRG0AmFCH5t87dflxk
219,0.620963,Heavenly Father,Isaiah Rashad,Cilvia Demo,0qLt7pQhm8LDxFgI0YXdlY
8,0.620397,Small Victories,Warm Brew,Small Victories,6RMvgOHWSj0YnB9SHvY9cP
80,0.619377,Days Passed Me By,Kooley High,Presents...David Thompson,1wMRm72vhW96O6uPkXkMQT


In [12]:
create_new_model(playlist_features)

New classification model created.


In [67]:
user_preference_playlist.to_csv('spotify_stuff.csv', sep = ',')

In [7]:
user_preference_playlist = pd.read_csv('spotify_stuff.csv', sep = ',')
user_preference_playlist.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [35]:
analysis_results = predict_scores(playlist_features)
rank(analysis_results)

,scores,name,artist,album,track_id
28,0.131832,Suede,NxWorries,Yes Lawd!,7kFxZ5ckpMi2RhN1WH8cn2
160,0.130640,Stoney,Saba,Bucket List Project,0HU8CeTestQjoRF2ekaLqx
46,0.129160,The Only One,Blu & Exile,Give Me My Flowers While I Can Still Smell Them,53GyUog0e8fbctqyJieLS2
37,0.127096,Alone,Kooley High,Heights,74ekur2cSq38RshtjzNMAX
179,0.126630,You Stressin',Bishop Nehru,You Stressin',60NwuLLD9fpWZcDIfjv1NS
134,0.126620,Come Correct,Gifted Gab,Come Correct,2iBJmZizVROOBnQx7T08iR
158,0.125656,U 47,Your Old Droog,Your Old Droog,1otYIRG0AmFCH5t87dflxk
36,0.125605,Spiders,Illa J,Spiders,59XfkUPhqn61BhHx21HYjL
219,0.125327,Heavenly Father,Isaiah Rashad,Cilvia Demo,0qLt7pQhm8LDxFgI0YXdlY
193,0.124045,Jazzhop,Kero One,Kero & Azure,0qRQetj8L2LzKMdMczzR9w


In [27]:
analysis_results.head()

,track_id,artist_id,name,album,artist,release_date,length,popularity,key,mode,...,danceability,energy,valence,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,scores
0,7eLpkw9x3p88Fga5drnvqS,2J2V1PxHVZ1ANtT8Y2UdTS,Blah Loops,Songs Made While High,Like,2016-09-30,3.741667,0.27,1,1,...,0.688,0.743,0.572,0.002740,0.116,-6.880,0.111,88.513,4,0.104276
1,2TTqgKHQwSkvxD6fI25nWK,0nvdwVbj7NT1WL9P8JowLD,Hands Up!,Hands Up!,blackwave.,2017-08-18,3.665000,0.13,0,1,...,0.620,0.443,0.581,0.000043,0.119,-8.145,0.375,87.423,4,0.100227
2,6bvV5L5afKykg819xAIJWt,7bTL2or1gi4VAjY7XzoK8e,First Person,Time Being (Deluxe Edition),CYNE,2017-09-08,2.837933,0.48,1,1,...,0.686,0.800,0.467,0.000549,0.286,-7.279,0.216,96.480,4,0.103823
3,6NXqSpbK8xkfOZRTVhC5nr,1mOiWC7OH9ANUtt3vd0A10,Smile,Smile,Shuko,2017-10-06,2.864883,0.00,9,1,...,0.551,0.675,0.374,0.000003,0.139,-4.668,0.345,101.952,5,0.093665
4,5Wd3qtcrfnNc0zuQLbUsES,4pCZmnefAzcD4fVSkcVEYf,Polaroid,Polaroid,Melanin 9,2017-10-16,3.342317,0.00,7,1,...,0.749,0.828,0.495,0.000001,0.101,-7.785,0.322,77.648,4,0.086845


In [25]:
rank(analysis_results)[:1]['track_id'].tolist()[0]

'7kFxZ5ckpMi2RhN1WH8cn2'

In [58]:
ranked_playlist = rank(analysis_results)
slice_top = ranked_playlist.loc[ranked_playlist['scores'] >= 0.1]
top_bois = slice_top['track_id'].tolist()

In [60]:
random_bois = random.sample(top_bois, 5)

In [80]:
seed_tracks = rank(eif_playlist)[:5]['track_id'].tolist()
seed_rec = sp.recommendations(seed_tracks = seed_tracks, limit = 100)

In [92]:
rec_id_list = parse_rec_id_list(seed_rec)
red_id_features = parse_features(rec_id_list)

In [100]:
# eif version
eif_id_features = red_id_features
rec_eif_id_features = preprocess(eif_id_features).to_numpy()
S1 = F1.compute_paths(X_in=rec_eif_id_features)
eif_id_features['scores'] = 1 - S1
rank(eif_id_features)

,scores,name,artist,album,track_id
0,0.597386,Maybe One Day,Blu & Exile,Give Me My Flowers While I Can Still Smell Them,5hYEaLnayI7QbBV9vXyu5Y
88,0.594522,That Chicken,DyMe-A-DuZiN,Crown Fried,3rrs2kN4bhqSwHMLID2YQe
72,0.585579,Roses,BJ The Chicago Kid,1123 (Deluxe Edition),18aTO0JlNoGaFx6hVNbTxB
24,0.579852,All Caps,Madvillain,Madvillainy,21O0XXPEWPtePt5RMY93Ob
64,0.562587,Already,Kemba,Negus,554hB35yOQCC7bkiiDNMbY
16,0.551913,Walk On By,Thundercat,Drunk,6fkL9CjWmjeV6vfDxTp9OZ
8,0.543643,Dear Raleigh,Kooley High,Presents...David Thompson,0siQP9KiIRRrgvlDJQG7Us
56,0.513506,Do It All,Ethereal,Catalyst,3BbHzSBSiPtk7qH0GH1elP
40,0.510143,Amnesia,Blu,Her Favorite Colo(u)r,1wNtOxs2WXWgGwoHBb7LNG
32,0.484451,Ain't a Damn Thing Change,Statik Selektah,8,6Eg7DdyXfi7Fv8y0xVQl8X


In [82]:
rec_results = predict_scores(red_id_features)
rank(rec_results)

,scores,name,artist,album,track_id
32,0.127445,Ain't a Damn Thing Change,Statik Selektah,8,6Eg7DdyXfi7Fv8y0xVQl8X
2,0.123372,Gang Signs (feat. ScHoolboy Q),Freddie Gibbs,Gang Signs (feat. ScHoolboy Q),6snYsavG9vq3C7RGN98dst
12,0.120204,It Ain't Easy (Bonus Track),Chuuwee,Amerikkas Most Blunted,24zNLxVrb2h8hDL8FU6hju
31,0.118137,All the Way,Strange Fruit Project,Soul Travelin',1H7L66uMYxG6MpB2r4DklO
76,0.117666,Meditate (feat. J.I.D.),EARTHGANG,Rags,0Eqm7hD828cATBLUx2fJox
35,0.116247,BLACK BALLOONS | 13LACK 13ALLOONZ [FEAT. TWELV...,Denzel Curry,TA13OO,5mexbTuWx9d8DPZk4sDGF4
96,0.115485,Passin' Me By,The Pharcyde,Bizarre Ride II The Pharcyde,4G3dZN9o3o2X4VKwt4CLts
8,0.110362,Dear Raleigh,Kooley High,Presents...David Thompson,0siQP9KiIRRrgvlDJQG7Us
99,0.109870,Thieves In The Night,Black Star,Mos Def & Talib Kweli Are Black Star,7tx1TUJrT6qxXFXAELqbev
13,0.109285,40 Winks,Jonwayne,40 Winks,0BF6hJi4yDcI6sylcGH2TS


In [102]:
frames = [eif_playlist, eif_id_features]
pull_list = pd.concat(frames, ignore_index = True)
pull_list = pull_list.drop_duplicates(subset = 'track_id')
len(pull_list)

330

In [106]:
input_track = rank(eif_playlist)[:1]['track_id'].tolist()[0]
input_playlist = pull_list
new_playlist = generate_playlist(input_track, input_playlist)
new_playlist

,track_id,name,artist,album
0,0HU8CeTestQjoRF2ekaLqx,Stoney,Saba,Bucket List Project
1,74ekur2cSq38RshtjzNMAX,Alone,Kooley High,Heights
2,0qLt7pQhm8LDxFgI0YXdlY,Heavenly Father,Isaiah Rashad,Cilvia Demo
3,2LaY7tGSS6cahRMDe10Q6h,Networth,Oh No,3 Dimensional Prescriptions
4,5UFtKaL5KBumqv4tpgdm0p,Laputa (Taylor McFerrin Remix feat. Anderson ....,Hiatus Kaiyote,"Recalibrations, Vol. 1"
5,7dDfQs8HUz5JHZdaEAF8gX,Memory Lane,Elzhi,Elmatic
6,560CdkdPdiqDr9ccd0cQyv,Cookie Chips,Rejjie Snow,Baw Baw Black Sheep
7,1wMRm72vhW96O6uPkXkMQT,Days Passed Me By,Kooley High,Presents...David Thompson
8,7hjqxwod60R36qbOSeqp3B,Nothin' at All,Common Market,Tobacco Road
9,62bvu8hO9tPS0DQ3WzYYwp,Robots,EARTHGANG,Robots


In [107]:
df = pd.merge(new_playlist, eif_playlist, on=['track_id'], how='left', indicator='Exist')
df['Exist'] = np.where(df.Exist == 'both', True, False)

In [108]:
df[['track_id', 'name_x','artist_x','album_x','Exist']]

,track_id,name_x,artist_x,album_x,Exist
0,0HU8CeTestQjoRF2ekaLqx,Stoney,Saba,Bucket List Project,True
1,74ekur2cSq38RshtjzNMAX,Alone,Kooley High,Heights,True
2,0qLt7pQhm8LDxFgI0YXdlY,Heavenly Father,Isaiah Rashad,Cilvia Demo,True
3,2LaY7tGSS6cahRMDe10Q6h,Networth,Oh No,3 Dimensional Prescriptions,False
4,5UFtKaL5KBumqv4tpgdm0p,Laputa (Taylor McFerrin Remix feat. Anderson ....,Hiatus Kaiyote,"Recalibrations, Vol. 1",False
5,7dDfQs8HUz5JHZdaEAF8gX,Memory Lane,Elzhi,Elmatic,True
6,560CdkdPdiqDr9ccd0cQyv,Cookie Chips,Rejjie Snow,Baw Baw Black Sheep,False
7,1wMRm72vhW96O6uPkXkMQT,Days Passed Me By,Kooley High,Presents...David Thompson,True
8,7hjqxwod60R36qbOSeqp3B,Nothin' at All,Common Market,Tobacco Road,True
9,62bvu8hO9tPS0DQ3WzYYwp,Robots,EARTHGANG,Robots,True


In [109]:
print("We've added " + str(df.Exist.value_counts()[False]) + " new reccomendations.")

We've added 4 new reccomendations.


In [110]:
refresh()
new_ai_list = sp.user_playlist_create(user =user_id,
                                      name = "tomwouldy AI list",
                                      public = True,
                                      collaborative = False,
                                      description = 'generated by AI')

In [111]:
id_ai_list = new_ai_list['id']

In [112]:
ai_track_list = new_playlist['track_id'].to_list()

In [113]:
sp.user_playlist_add_tracks(user = user_id, playlist_id = id_ai_list, tracks = ai_track_list, position=None)

{'snapshot_id': 'Myw1NmEyMTBmZGU4MTI3MWUxMzgwOTA1N2YxNDU2NDdhZDhlYzU4ZjVj'}